In [3]:
import csv
import ccxt
import json
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime

In [14]:
class FetchFromFTX():
    
    def __init__(self, api_key, api_secret, subaccount=None, symbol=None, post_only=False):
        self.api_key = api_key
        self.api_secret = api_secret
        self.subaccount = subaccount
        self.symbol = symbol
        self.post_only = post_only
        self.exchange = ccxt.ftx({
            'apiKey' : api_key, 
            'secret' : api_secret,
            'enableRateLimit' : True})
        
        if self.subaccount == None:
            print('-' * 16)
            print('FTX Main Account')
            print('-' * 16)
            
        else:
            self.exchange.headers = {
                'FTX-SUBACCOUNT': self.subaccount,}
            print('-' * 26)
            print(f'FTX Sub Account : {self.subaccount}')
            print('-' * 26)
            
    def fetchPrice(self):
        pair = self.symbol
        try:
            price = float(self.exchange.fetch_ticker(pair)['last'])
        except ccxt.NetworkError as e:
            price = float(self.exchange.fetch_ticker(pair)['last'])
        except ccxt.ExchangeError as e:
            price = float(self.exchange.fetch_ticker(pair)['last'])
        except Exception as e:
            price = float(self.exchange.fetch_ticker(pair)['last'])
        return price 
    
    def executeOrder(self, types, side, unit, price):
        # (pair, "limit", "buy", 0.001, 51000)
        pair = self.symbol
        
        try:
            order = self.exchange.create_order(pair, types, side, unit, price, params={'postOnly':self.post_only}) 
            print(f'{side} {types}: {pair} price: {price} unit: {unit}') 
            
        except:
            print('Pending failed')
            

    def fetchPending(self):
        pair = self.symbol
        dfPending = pd.DataFrame(self.exchange.fetch_open_orders(pair, since=None, limit=1000, params={}), 
                     columns=['id', 'timestamp', 'datetime', 'symbol', 'side', 'price', 'amount', 'status', 'filled'])
        return dfPending
    
    def fetchTradeHistory(self):
        pair = self.symbol
        dfHist = pd.DataFrame(self.exchange.fetch_my_trades(pair, since=None, limit=1000, params={}),
                     columns=['id', 'timestamp', 'datetime', 'symbol', 'side', 'price', 'amount', 'cost', 'fee'])
        
        #ftx.fetchOrderHistory()['fee'].iloc[0]['cost'] 
        return dfHist
        
    def cancelAllOrder(self):
        order_id = self.fetchPending()['id']
        for i in order_id:
            try:
                self.exchange.cancel_order(i) 
                print(f'Order ID : {i} has been cancelled')
                
            except:
                print('Cancel failed')
                
        
    def fetchBalance(self):
        return self.exchange.fetch_balance(params={})
    
    def fetchOrder(self, order_id):
        # d = 'status' ,'filled' ,'price etc', 'side'.
        return self.exchange.fetch_order(order_id)
    
    def OHLCV(self, tf, limit=None):
        #tf = '15s', '1m', '5m', '15m', '1h', '4h', '1d'
        pair = self.symbol
        ohlcv = self.exchange.fetch_ohlcv(pair, timeframe=tf, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
        return df
    
    def minimumSize(self):
        pair = self.symbol
        min_size = self.exchange.fetch_ticker(pair)['info']['minProvideSize']
        return min_size
    

In [18]:
ftx = FetchFromFTX(api_key='j2VxGzorpxPXmAcHgI2X4JVlqo_XI9OjzlZ7KEtS',
                   api_secret='84GNy7cbZZd6eXBlJwRjyqf64ABkeeUup5F8W1S0',
                   subaccount='XRP001',
                   symbol='XRP/USD',
                   post_only=True)

--------------------------
FTX Sub Account : XRP001
--------------------------


In [17]:
ftx.executeOrder('limit','buy', 1, 0.1)

Pending failed


In [18]:
ftx.cancelAllOrder()

In [8]:
ftx.fetchPending()

,id,timestamp,datetime,symbol,side,price,amount,status,filled


In [10]:
ftx.fetchTradeHistory()

,id,timestamp,datetime,symbol,side,price,amount,cost,fee
0,2466931131,1622567405670,2021-06-01T17:10:05.670Z,XRP/USDT,buy,0.9950,1.0,0.9950,"{'cost': 0.00019, 'currency': 'XRP', 'rate': 0..."
1,2525068277,1623003266043,2021-06-06T18:14:26.043Z,XRP/USDT,sell,0.9375,1.0,0.9375,"{'cost': 0.000178125, 'currency': 'USDT', 'rat..."
2,2525151371,1623004475602,2021-06-06T18:34:35.602Z,XRP/USDT,buy,0.9350,1.0,0.9350,"{'cost': 0.00019, 'currency': 'XRP', 'rate': 0..."


In [19]:
ftx.fetchTradeHistory()

,id,timestamp,datetime,symbol,side,price,amount,cost,fee
0,2834170953,1625591719996,2021-07-06T17:15:19.996Z,XRP/USD,buy,0.656000,3.0,1.968000,"{'cost': 0.00057, 'currency': 'XRP', 'rate': 0..."
1,2838068730,1625632274602,2021-07-07T04:31:14.602Z,XRP/USD,sell,0.671650,3.0,2.014950,"{'cost': 0.0003828405, 'currency': 'USD', 'rat..."
2,2846382781,1625711418229,2021-07-08T02:30:18.229Z,XRP/USD,buy,0.637900,7.0,4.465300,"{'cost': 0.00133, 'currency': 'XRP', 'rate': 0..."
3,2848850757,1625730245070,2021-07-08T07:44:05.070Z,XRP/USD,buy,0.622700,4.0,2.490800,"{'cost': 0.00076, 'currency': 'XRP', 'rate': 0..."
4,2856234487,1625797042673,2021-07-09T02:17:22.673Z,XRP/USD,buy,0.606550,4.0,2.426200,"{'cost': 0.00076, 'currency': 'XRP', 'rate': 0..."
5,2856999414,1625804259388,2021-07-09T04:17:39.388Z,XRP/USD,sell,0.622275,3.0,1.866825,"{'cost': 0.00035469675, 'currency': 'USD', 'ra..."
6,2859252958,1625829876668,2021-07-09T11:24:36.668Z,XRP/USD,buy,0.607425,3.0,1.822275,"{'cost': 0.00057, 'currency': 'XRP', 'rate': 0..."
7,2859933672,1625836083440,2021-07-09T13:08:03.440Z,XRP/USD,sell,0.623000,3.0,1.869000,"{'cost': 0.00035511, 'currency': 'USD', 'rate'..."
8,2860796278,1625842305552,2021-07-09T14:51:45.552Z,XRP/USD,sell,0.633925,3.0,1.901775,"{'cost': 0.00036133725, 'currency': 'USD', 'ra..."
9,2867733819,1625924862509,2021-07-10T13:47:42.509Z,XRP/USD,buy,0.619125,3.0,1.857375,"{'cost': 0.00057, 'currency': 'XRP', 'rate': 0..."
